In [1]:
import pandas as pd
from pandas import Series, DataFrame

In [13]:
pd.options.display.max_rows = 10

### 6.2 binary data foramts

In [2]:
frame = pd.read_csv('examples/ex1.csv')

In [3]:
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
frame.to_pickle('examples/frame_pickle')

In [5]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


use HDF5 formats

In [6]:
import numpy as np
frame = pd.DataFrame({'a': np.random.randn(100)})

创建HDF5文件文件，`HSFStore`方法需要用到`tables`模块，没有安装的话需要执行`pip3 install tables`

In [8]:
store = pd.HDFStore('mydata.h5')

In [9]:
store['obj1'] = frame

In [10]:
store['obj1_col'] = frame['a']

In [11]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [14]:
store['obj1']

,a
0,-0.279113
1,-0.822993
2,-1.097854
3,0.431529
4,1.598631
...,...
95,0.721054
96,0.992600
97,-1.996451
98,-1.393521


In [15]:
store.put('obj2', frame, format='table')

In [16]:
store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,-0.040620
11,-0.084617
12,1.025473
13,1.233380
14,0.528772
15,0.067294


In [17]:
store.close()

In [20]:
frame.to_hdf('mydata.h5', 'obj3', format='table')

In [21]:
pd.read_hdf('mydata.h5', 'obj3', where=['index <= 5'])

,a
0,-0.279113
1,-0.822993
2,-1.097854
3,0.431529
4,1.598631
5,-1.262907


读取Microsoft Excel文件

In [22]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')

In [23]:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [24]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')

In [25]:
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [26]:
writer = pd.ExcelWriter('examples/ex2.xlsx')

In [27]:
frame.to_excel(writer, 'Sheet1')

In [28]:
writer.save()

### 6.3 Web APIs 交互

In [29]:
import requests

In [30]:
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'

In [31]:
resp = requests.get(url)

In [32]:
resp

<Response [200]>

In [33]:
data = resp.json()

In [34]:
data[0]['title']

'PERF: do not instantiate IndexEngine for standard lookup over RangeIndex'

In [35]:
issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])

In [36]:
issues

,number,title,labels,state
0,27119,PERF: do not instantiate IndexEngine for stand...,[],open
1,27118,Plotting ExtensionArrays,[],open
2,27117,Make pandas.to_parquet handles partition colum...,[],open
3,27116,Bump python_requires to 3.5.2,"[{'id': 129350, 'node_id': 'MDU6TGFiZWwxMjkzNT...",open
4,27114,Decoupled more xlrd reading tests from openpyxl,"[{'id': 49254273, 'node_id': 'MDU6TGFiZWw0OTI1...",open
...,...,...,...,...
25,27076,Blacken the code base,"[{'id': 77550281, 'node_id': 'MDU6TGFiZWw3NzU1...",open
26,27075,Groupby ignores unobserved combinations when p...,[],open
27,27073,ENH: Json fill_value for missing fields,"[{'id': 49379259, 'node_id': 'MDU6TGFiZWw0OTM3...",open
28,27072,[POC] CLN: use ExtensionBlock for datetime tz ...,"[{'id': 849023693, 'node_id': 'MDU6TGFiZWw4NDk...",open


### 6.4 数据库交互

In [37]:
import sqlite3

In [38]:
query = """
create table test (
a varchar(20),
b varchar(20),
c real,
d integer
);
"""

In [39]:
con = sqlite3.connect('mydata.sqlite')

In [40]:
con.execute(query)

In [41]:
con.commit()

In [42]:
data = [('Atlanta', 'Georgia', 1.25, 6),
       ('Tallahassee', 'Florida', 2.6, 3),
       ('Sacramento', 'California', 1.7, 5)]

In [43]:
stmt = 'insert into test values(?, ?, ?,?)'

下面执行了插入数据sql，但是需要提交提交操作，使用`con.commit()`

In [44]:
con.executemany(stmt, data)

In [59]:
con.commit()

In [45]:
cursor = con.execute('select * from test')

In [46]:
rows = cursor.fetchall()

In [47]:
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [48]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [49]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [50]:
import sqlalchemy as sqla

In [51]:
db = sqla.create_engine('sqlite:///mydata.sqlite')

In [58]:
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
